Création de 3 fichiers de production
prodA pour le pays A, prodB pour le pays B, prodC pour le pays C

Les fichiers contiennent en réalité les quantité d'électricité produites (éolien + solaires) du programme ENR_PL_classique découpées en 3 trimestres.

In [ ]:
import numpy as np
import pandas as pd

### Choix

In [ ]:
N = 3 # Nombre de fichiers de production à créer (ie de de pays dans la simulation)

In [ ]:
# Chargement des données
df_solar = pd.read_csv('solar.csv')
df_wind = pd.read_csv('wind_onshore.csv')
df_demand = pd.read_csv('demand2050_ADEME.csv', header=None)
df_demand.columns = ["heures", "demande"]

# Calcul consommation annuelle
annual_demand = df_demand['demande'].sum()

# Calcul production potentielle pour 1GW installé
annual_solar_per_gw = df_solar['facteur_charge'].sum()
annual_wind_per_gw = df_wind['facteur_charge'].sum()

# Pour avoir 100% de surproduction et un mix 50-50
target_production = annual_demand * 3.5
wind_capacity = target_production/(2 * annual_wind_per_gw)
solar_capacity = target_production/(2 * annual_solar_per_gw)


Capacités requises: éolien = 297.6GW, solaire = 539.7GW


Création de 3 bdd de production différentes, fictives, pour les pays A, B et C

In [ ]:
nbHeures = len(df_demand)  # nombre d'heures
T = nbHeures//N

solar_profileA = df_solar['facteur_charge'].values[:T]
wind_profileA = df_wind['facteur_charge'].values[:T]
demandA = df_demand['demande'].values[:T]

solar_profileB = df_solar['facteur_charge'].values[T:2*T]
wind_profileB = df_wind['facteur_charge'].values[T:2*T]
demandB = df_demand['demande'].values[T:2*T]

solar_profileC = df_solar['facteur_charge'].values[2*T:3*T]
wind_profileC = df_wind['facteur_charge'].values[2*T:3*T]
demandC = df_demand['demande'].values[2*T:3*T]

# Production ENR horaire
prodA = {t: wind_capacity * wind_profileA[t] + solar_capacity * solar_profileA[t] for t in range(len(solar_profileA))}
prodB = {t: wind_capacity * wind_profileB[t] + solar_capacity * solar_profileB[t] for t in range(len(solar_profileB))}
prodC = {t: wind_capacity * wind_profileC[t] + solar_capacity * solar_profileC[t] for t in range(len(solar_profileC))}

# Création de la base de données
dfA = pd.DataFrame(list(prodA.items()), columns=['Heure', 'Production_Elec'])
dfA.to_csv('prodA.csv', index=False, encoding='utf-8')
dfB = pd.DataFrame(list(prodB.items()), columns=['Heure', 'Production_Elec'])
dfB.to_csv('prodB.csv', index=False, encoding='utf-8')
dfC = pd.DataFrame(list(prodC.items()), columns=['Heure', 'Production_Elec'])
dfC.to_csv('prodC.csv', index=False, encoding='utf-8')
